# 列生成求解下料问题
限制性主问题（Restricted Master Problem）为最小化使用的棒材原料的数量

$$
\begin{align*}
    \min z = &x_1 +x_2 +x_3 +x_4 +x_5 \text{(RMP)}\\
        &8x_1+6x_2+6x_3+5x_4+5x_5 \ge 46\\
        &x_2 +x_4 +2x_5 \ge 22\\
        &x_3 +x_4 \ge 43\\
        &x_1,\; x_2, \;x_3,\; x_4, \;x_5 \ge 0 \text{  and integer}
\end{align*}
$$

子问题（Subproblem） 为

$$
\begin{align*}
    \min z = &1 − y(a_1, a_2, a_3) \text{(SP)}\\
        &10a_1 + 11a_2 + 19a_3 \le 80\\
        &a_1,\; a_2, \;a_3 \ge 0 \text{  and integer}
\end{align*}
$$

主问题先松弛为LP，为子问题提供对偶变量，子问题为主问题提供新列，直到子问题无法产生新列（子问题的z大于0）为止，最后求解完整主问题

In [1]:
from gurobipy import *

# 所需棒料的种类数
itemNum = 3

# 原材料的长度
Length = 80

# 所需棒料的宽度与需求
l = {0:10, 1:11, 2:19}
d = {0:46, 1:43, 2:22}

# 建立子问题
SP = Model('subproblem')
SPvars = SP.addVars(itemNum, obj=-1, vtype=GRB.INTEGER, name='w')  #设置目标函数中的系数分别为 -a1-a2-a3
SP.addConstr(SPvars.prod(l) <= Length)  #设置约束 10 a1 + 11 a2 + 19 a3 <= 80
SP.write('subproblem.lp')
SP.update()

# 建立初始主问题

# 系数矩阵
coef = [  [8,6,6,5,5],
       [0,1,0,1,2],
       [0,0,1,1,0],]

# 右端项
rhs = [46, 22, 43]

MP = Model('master_problem')

x = {}
for i in range(5):
    x[i] = MP.addVar(obj=1, vtype=GRB.CONTINUOUS, name='x_' + str(i+1))
    
cons = {}
for i in range(3):
    exp = LinExpr()
    for j in range(5):
        exp.addTerms(coef[i][j], x[j])
    cons[i] = MP.addConstr(exp>=rhs[i], name = 'c_'+str(i+1))
    
MP.write('initial_MP.lp')
# 用初始变量与初始列建立主问题
MP.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 3 rows, 5 columns and 10 nonzeros
Model fingerprint: 0xb319440f
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+01]
Presolve time: 0.01s
Presolved: 3 rows, 5 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.975000e+01   0.000000e+00      0s
       2    4.3000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.300000000e+01


In [2]:
# 不断循环直到找到感兴趣的列
SP.Params.OutputFlag = 0
MP.Params.OutputFlag = 0
Iter = 0
eps = -0.0001  # tolerance
while( MP.Status == GRB.OPTIMAL ):
    '''
    reduced cost is c_i - pi * A_{*i}
    - A_{*i} in our case is the proposed solution in SP
    - pi comes from the duals in MP
    - c_i is the coefficient of variable in obj function
    '''
    pi = {i: -cons[i].Pi for i in range(itemNum)}
    SP.setObjective(SPvars.prod(pi))
    SP.optimize()
    
    if( SP.Status != GRB.OPTIMAL ):
        raise('Unexpected optimization status')

    if( 1 + SP.ObjVal > eps ):     # 因为上面的SP目标函数缺了常数1，这里补上
        break
        
    if(Iter % 10 == 0):
        print('Iteration MasterValue PricingValue')
        
    print('%8d %12.5g %12.5g' % (Iter, MP.ObjVal, SP.ObjVal))
    Iter += 1
    
    # Using solution, build new variable
    col = Column()
    for j in range(itemNum):
        col.addTerms(SPvars[j].X, cons[j])
    MP.addVar(obj=1, column=col, name = 'new_x' + str(Iter))
    MP.optimize()
    
MP.write('final_MP.lp')
rootbound = MP.ObjVal
MP.Params.OutputFlag = 1

Mvars = MP.getVars()
for v in Mvars:
    v.VType = GRB.INTEGER # 将变量的类型改成整型。
MP.optimize()

print('Obj value =', MP.ObjVal)

for v in MP.getVars():
    print('{} = {}'.format(v.VarName, v.X))

Iteration MasterValue PricingValue
       0           43           -4
       1        21.75         -3.5
       2       19.479      -1.0357
Set parameter OutputFlag to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 3 rows, 8 columns and 14 nonzeros
Model fingerprint: 0xffbdbd9e
Variable types: 0 continuous, 8 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+01]
Found heuristic solution: objective 25.0000000
Presolve time: 0.00s
Presolved: 3 rows, 8 columns, 14 nonzeros
Variable types: 0 continuous, 8 integer (0 binary)

Root relaxation: objective 1.925000e+01, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node 